In [1]:
import scipy.io
import pandas as pd
import numpy as np 
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score, cross_val_predict
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, roc_auc_score
import scipy.stats as stats
from mat_preproc import preproc

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set_style("darkgrid")

In [2]:
# the mat file is generated by CRM_SN_vs_MN_imbalLDA_1450ms_order_proj.m

#Load in the MATLAB file with all the data
data = scipy.io.loadmat('data_CRM_SN_vs_MN_imbalLDA_order_proj_1.mat')
# data = scipy.io.loadmat('data_CRM_SN_vs_MN_imbalLDA_order_proj_2.mat')

print(data.keys())

dict_keys(['__header__', '__version__', '__globals__', 'user_class_min_1', 'user_feat_1', 'user_prob_1', 'user_resp_1', 'user_source_1', 'user_tr_order_1', 'user_train_prob_1', 'user_weights_1'])


In [3]:
#Had to index them like this because there was an unnecessary dimension 

# user trail order
tr_order = data['user_tr_order_1'][0]

# projection scores
proj_score = data['user_prob_1'][0]

# source and response label
source_label = data['user_source_1'][0]
resp_label = data['user_resp_1'][0]

# features, group the channels and average over the windows,
# those are called features, and we use the features for training
behav_feat = data['user_feat_1'][0]

# Encodings for Each Label

## *source information*

1. SC (Source Correct)
2. CR (Correct Rejection)
3. SI (Source Incorrect)
4. Miss 
5. FA  (False Alarm)

## *label for the source response*

1. RS (Remember Source)
2. RO (Remember Other)
3. F (Familiarity)
4. MN (Maybe New) 
5. SN (Sure New)


In [4]:
# class attributes
source_info = ["SC", "CR", "SI", "M", "FA"]
response_info = ["RS", "RO", "F", "MN", "SN"]

# the x-axis on the projection graph
x_axis = [(1, 1), (3, 1), (5, 1), (1, 2), (5, 2), (1, 3), (3, 3), (5, 3), (4, 4), (2, 4), (4, 5), (2, 5)]

In [5]:
for source, resp in x_axis:
    print(f"{source_info[source-1]}-{response_info[resp-1]}")

SC-RS
SI-RS
FA-RS
SC-RO
FA-RO
SC-F
SI-F
FA-F
M-MN
CR-MN
M-SN
CR-SN


# Preproc

In [6]:
# multiple group
file_path = "data_CRM_SN_vs_MN_imbalLDA_order_proj_1.mat"
# file_path = "data_CRM_SN_vs_MN_imbalLDA_order_proj_2.mat"
data_preproc = preproc(file_path, 1)
pos1, neg1 = data_preproc.filter_index(2,5,2,4)
pos2, neg2 = data_preproc.filter_index(4,5,4,4)
pos_idx, neg_idx = data_preproc.merge_two_class(pos1, neg1, pos2, neg2)
X, y, subject = data_preproc.get_data_by_index(pos_idx, neg_idx)
X.shape

(3813, 72)

In [7]:
subject.shape

(3813,)

In [8]:
y.shape

(3813,)

# Trainning - SN v.s. MN

# Testing the multiclass projection onto this classifier

In Keuida's code, the projection are calculated within fold. e.g, each participant's projection is based the LDA trained on the rest of the participants' data, instead of fitting it directly. Therefore, we might need to do the manual fitting in each fold of the LOSO.

In [ ]:
projections = []

curr = 0
for train_idx, test_idx in logo.split(X, y, subject):
    # select out the id of the participant that we left out
    participant = int(subject[curr])
    curr += len(test_idx)
    # this LOSO follows the sequence of ppl presented in subject
    X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx]
    LDA.fit(X_train, y_train)
    # project the whole data of this complete participant
    X_subject = data_preproc.get_data_by_participant(participant)
    projection = LDA.transform(X_subject)
    projections.append(projection)
projections = np.concatenate(projections)

In [ ]:
x_name = np.array([f"{source_info[s-1]}-{response_info[r-1]}" for s, r in x_axis])

In [ ]:
pos_idx = [10, 11]
neg_idx = [8, 9]

summary_stats = []

for source, response in x_axis:
    idx = data_preproc.filter_index_single_class(source, response, include_left_out=False)
    proj = projections[idx]
    mean = proj.mean()
    std = proj.std() / np.sqrt(len(proj)) * 1.96
    summary_stats.append((mean, std))

summary_stats = np.array(summary_stats)

plt.errorbar(x_name, summary_stats[:,0], yerr = summary_stats[:,1],
             fmt='o', capsize=3, c = 'black')
# pos
plt.errorbar(x_name[pos_idx], summary_stats[pos_idx,0],
             yerr = summary_stats[pos_idx,1] , fmt='o', capsize=3, c = 'lime', label="neg class")
# neg
plt.errorbar(x_name[neg_idx], summary_stats[neg_idx,0],
             yerr = summary_stats[neg_idx,1] , fmt='o', capsize=3, c = 'red', label="pos class")

_ = plt.xticks(rotation=45)
plt.legend()

# Replicate Acc based on Kueida's Method

In [18]:
def cal_acc_balanced(clf, trail_num):
    file_path = f"data_CRM_SN_vs_MN_imbalLDA_order_proj_{trail_num}.mat"
    data_preproc = preproc(file_path, trail_num)
    if clf == "SN_MN":
        pos1, neg1 = data_preproc.filter_index(2,5,2,4)
        pos2, neg2 = data_preproc.filter_index(4,5,4,4)
    elif clf == "F_CR":
        pos1, neg1 = data_preproc.filter_index(1,3,2,4)
        pos2, neg2 = data_preproc.filter_index(3,3,2,5)
    else:
        raise ValueError("Unknown Classifier. Should be either `SN_MN` or `F_CR`")
    pos_idx, neg_idx = data_preproc.merge_two_class(pos1, neg1, pos2, neg2)
    X, y, subject = data_preproc.get_data_by_index(pos_idx, neg_idx)

    logo = LeaveOneGroupOut()

    scores = []

    for train_idx, test_idx in logo.split(X, y, subject):
        X_train, y_train = X[train_idx,:], y[train_idx]
        X_test, y_test = X[test_idx,:], y[test_idx]
        LDA = LinearDiscriminantAnalysis(shrinkage = None, solver = 'eigen')
        LDA.fit(X_train, y_train)
        # randomly drop datapoint to balance class
        pos_idx, neg_idx = np.arange(len(test_idx))[y_test == 1], np.arange(len(test_idx))[y_test != 1]
        pos_len, neg_len = len(pos_idx), len(neg_idx)
        acc = []
        for _ in range(10):
            if pos_len > neg_len:
                # when there are more positive class than negative
                # randomly drop positive class to equivalent the negative class
                pos_chosen = np.random.choice(pos_idx, neg_len, replace=False)
                neg_chosen = neg_idx
            else:
                pos_chosen = pos_idx
                neg_chosen = np.random.choice(neg_idx, pos_len, replace=False)
            filter_test_idx = np.concatenate([pos_chosen, neg_chosen])
            X_test_balanced, y_test_balanced = X_test[filter_test_idx, :], y_test[filter_test_idx]
            assert sum(y_test_balanced) == 0 # to check whether they are balanced class
            acc.append(LDA.score(X_test_balanced, y_test_balanced))
        acc = np.array(acc)
        scores.append(acc)
    scores = np.array(scores)
    return scores.mean()

In [21]:
accs = [
    [cal_acc_balanced("SN_MN", 1), cal_acc_balanced("SN_MN", 2)],
    [cal_acc_balanced("F_CR", 1),  cal_acc_balanced("F_CR", 2)]
]
accs

[[0.5318622226187755, 0.5571189904836364],
 [0.5158917770440503, 0.514788403907315]]

Acc reported in the literature is 0.5421

In [ ]:
file_path = "data_CRM_SN_vs_MN_imbalLDA_order_proj_2.mat"
data_preproc = preproc(file_path, 2)
# combine SN and MN
pos1, neg1 = data_preproc.filter_index(2,5,2,4)
pos2, neg2 = data_preproc.filter_index(4,5,4,4)

pos_idx, neg_idx = data_preproc.merge_two_class(pos1, neg1, pos2, neg2)
X, y, subject = data_preproc.get_data_by_index(pos_idx, neg_idx)

logo = LeaveOneGroupOut()

scores = []

for train_idx, test_idx in logo.split(X, y, subject):
    X_train, y_train = X[train_idx,:], y[train_idx]
    X_test, y_test = X[test_idx,:], y[test_idx]
    LDA = LinearDiscriminantAnalysis(shrinkage = "auto", solver = 'eigen')
    LDA.fit(X_train, y_train)
    # randomly drop datapoint to balance class
    pos_idx, neg_idx = np.arange(len(test_idx))[y_test == 1], np.arange(len(test_idx))[y_test != 1]
    pos_len, neg_len = len(pos_idx), len(neg_idx)
    acc = []
    for _ in range(100):
        if pos_len > neg_len:
            # when there are more positive class than negative
            # randomly drop positive class to equivalent the negative class
            pos_chosen = np.random.choice(pos_idx, neg_len, replace=False)
            neg_chosen = neg_idx
        else:
            pos_chosen = pos_idx
            neg_chosen = np.random.choice(neg_idx, pos_len, replace=False)
        filter_test_idx = np.concatenate([pos_chosen, neg_chosen])
        X_test_balanced, y_test_balanced = X_test[filter_test_idx, :], y_test[filter_test_idx]
        assert sum(y_test_balanced) == 0 # to check whether they are balanced class
        acc.append(LDA.score(X_test_balanced, y_test_balanced))
    acc = np.array(acc)
    scores.append(acc)
scores = np.array(scores)
print(scores.mean())

acc reported in the literature is 0.5653

# Test the AUROC

In [ ]:
data_preproc = preproc(file_path, 1)
# combine SN and MN
pos1, neg1 = data_preproc.filter_index(2,5,2,4)
pos2, neg2 = data_preproc.filter_index(4,5,4,4)

pos_idx, neg_idx = data_preproc.merge_two_class(pos1, neg1, pos2, neg2)
X, y, subject = data_preproc.get_data_by_index(pos_idx, neg_idx)

logo = LeaveOneGroupOut()

scores = []

y_pred_prob = []
y_true = []

for train_idx, test_idx in logo.split(X, y, subject):
    X_train, y_train = X[train_idx,:], y[train_idx]
    X_test, y_test = X[test_idx,:], y[test_idx]
    LDA = LinearDiscriminantAnalysis(shrinkage = 'auto', solver = 'eigen')
    LDA.fit(X_train, y_train)
    y_pred_prob.append(LDA.predict_proba(X_test)[:,1])
    y_true.append(y_test)
y_pred_prob, y_true = np.concatenate(y_pred_prob), np.concatenate(y_true)
roc_auc_score(y_true, y_pred_prob)

The literature report this number as 0.5564

In [ ]:
file_path = "data_CRM_SN_vs_MN_imbalLDA_order_proj_2.mat"

data_preproc = preproc(file_path, 2)
# combine SN and MN
pos1, neg1 = data_preproc.filter_index(2,5,2,4)
pos2, neg2 = data_preproc.filter_index(4,5,4,4)

pos_idx, neg_idx = data_preproc.merge_two_class(pos1, neg1, pos2, neg2)
X, y, subject = data_preproc.get_data_by_index(pos_idx, neg_idx)

logo = LeaveOneGroupOut()

scores = []

y_pred_prob = []
y_true = []

for train_idx, test_idx in logo.split(X, y, subject):
    X_train, y_train = X[train_idx,:], y[train_idx]
    X_test, y_test = X[test_idx,:], y[test_idx]
    LDA = LinearDiscriminantAnalysis(shrinkage = 'auto', solver = 'eigen')
    LDA.fit(X_train, y_train)
    y_pred_prob.append(LDA.predict_proba(X_test)[:,1])
    y_true.append(y_test)
y_pred_prob, y_true = np.concatenate(y_pred_prob), np.concatenate(y_true)
roc_auc_score(y_true, y_pred_prob)

The literature report this number as 0.5997

# Replicate the F vs. CR clf